In [16]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
import os
from tqdm import tqdm
import torch
from matplotlib import pyplot as plt
import time
import os
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, BatchSampler, RandomSampler
import librosa

%matplotlib inline

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [13]:
labels = pd.read_csv("trainingData.csv")
le = preprocessing.LabelEncoder()
le.fit(labels["Language"])
labels["Language"] = le.transform(labels["Language"])
labels.head()

,Sample Filename,Language
0,000kouqjfnk.mp3,56
1,000w3fewuqj.mp3,151
2,000ylhu4sxl.mp3,155
3,0014x3zvjrl.mp3,35
4,001xjmtk2wx.mp3,69


In [14]:
train_dirs = []
train_base_dir = "Training data"
for direc in os.listdir("Training data"):
  train_dirs.append(direc)

In [18]:
train_labels = []
train_data = []
pbar = tqdm(total=66176)
for i, direc in enumerate(train_dirs):
    label = labels[labels["Sample Filename"] == direc]["Language"].item()
    y, sr = librosa.load(train_base_dir + "/" + direc)
    train_data.append(librosa.feature.melspectrogram(y=y, sr=sr))
    train_labels.append(label)
    pbar.update(1)
pbar.close()    
labels = np.array(train_labels)
train_data = np.array(train_data)
np.save("train_data.npy", train_data)





  0%|          | 0/66176 [00:00<?, ?it/s]



  0%|          | 1/66176 [00:01<22:22:03,  1.22s/it]



  0%|          | 2/66176 [00:01<18:29:14,  1.01s/it]



  0%|          | 3/66176 [00:02<15:48:54,  1.16it/s]



  0%|          | 4/66176 [00:02<13:57:37,  1.32it/s]



  0%|          | 5/66176 [00:03<12:50:04,  1.43it/s]



  0%|          | 6/66176 [00:03<12:04:30,  1.52it/s]



  0%|          | 7/66176 [00:04<11:34:16,  1.59it/s]



  0%|          | 8/66176 [00:05<11:11:04,  1.64it/s]



  0%|          | 9/66176 [00:05<10:54:34,  1.68it/s]



  0%|          | 10/66176 [00:06<10:44:53,  1.71it/s]



  0%|          | 11/66176 [00:06<10:20:50,  1.78it/s]



  0%|          | 12/66176 [00:07<10:21:35,  1.77it/s]



  0%|          | 13/66176 [00:07<10:21:33,  1.77it/s]



  0%|          | 14/66176 [00:08<10:14:21,  1.79it/s]



  0%|          | 15/66176 [00:08<10:12:37,  1.80it/s]



  0%|          | 16/66176 [00:09<10:05:56,  1.82it/s]



  0%|          | 17/66176 [00:09<10:17:05,  1.79

  0%|          | 118/66176 [01:07<10:20:37,  1.77it/s]



  0%|          | 119/66176 [01:08<10:25:24,  1.76it/s]



  0%|          | 120/66176 [01:08<10:26:22,  1.76it/s]



  0%|          | 121/66176 [01:09<10:27:37,  1.75it/s]



  0%|          | 122/66176 [01:10<10:27:49,  1.75it/s]



  0%|          | 123/66176 [01:10<10:26:33,  1.76it/s]



  0%|          | 124/66176 [01:11<10:24:00,  1.76it/s]



  0%|          | 125/66176 [01:11<10:23:04,  1.77it/s]



  0%|          | 126/66176 [01:12<10:12:45,  1.80it/s]



  0%|          | 127/66176 [01:12<10:02:01,  1.83it/s]



  0%|          | 128/66176 [01:13<10:09:18,  1.81it/s]



  0%|          | 129/66176 [01:13<10:13:05,  1.80it/s]



  0%|          | 130/66176 [01:14<10:05:07,  1.82it/s]



  0%|          | 131/66176 [01:14<9:53:51,  1.85it/s] 



  0%|          | 132/66176 [01:15<10:05:18,  1.82it/s]



  0%|          | 133/66176 [01:16<10:05:15,  1.82it/s]



  0%|          | 134/66176 [01:16<9:54:59,  1.85it/s] 



  0%|         

  0%|          | 235/66176 [02:13<10:16:38,  1.78it/s]



  0%|          | 236/66176 [02:14<10:08:55,  1.80it/s]



  0%|          | 237/66176 [02:14<10:17:49,  1.78it/s]



  0%|          | 238/66176 [02:15<10:06:30,  1.81it/s]



  0%|          | 239/66176 [02:16<10:13:23,  1.79it/s]



  0%|          | 240/66176 [02:16<10:19:27,  1.77it/s]



  0%|          | 241/66176 [02:17<10:25:09,  1.76it/s]



  0%|          | 242/66176 [02:17<10:27:50,  1.75it/s]



  0%|          | 243/66176 [02:18<10:27:53,  1.75it/s]



  0%|          | 244/66176 [02:18<10:33:03,  1.74it/s]



  0%|          | 245/66176 [02:19<10:30:23,  1.74it/s]



  0%|          | 246/66176 [02:20<10:33:49,  1.73it/s]



  0%|          | 247/66176 [02:20<10:31:39,  1.74it/s]



  0%|          | 248/66176 [02:21<10:41:41,  1.71it/s]



  0%|          | 249/66176 [02:21<10:38:32,  1.72it/s]



  0%|          | 250/66176 [02:22<10:36:21,  1.73it/s]



  0%|          | 251/66176 [02:23<10:33:43,  1.73it/s]



  0%|         

  1%|          | 352/66176 [03:20<10:18:18,  1.77it/s]



  1%|          | 353/66176 [03:20<10:08:15,  1.80it/s]



  1%|          | 354/66176 [03:21<10:19:34,  1.77it/s]



  1%|          | 355/66176 [03:22<10:21:38,  1.76it/s]

KeyboardInterrupt: 

In [6]:
# np.save("labels.npy", labels)

In [7]:
labels = np.load("labels.npy")

In [9]:
data = np.load("traindata.npy")
# np.random.shuffle(data)

In [10]:
data.shape
max(labels)

175

In [11]:
test_size = 20000
train_size = data.shape[0] - test_size
print(train_size)
train = data[:train_size, :, :]
train_label = labels[:train_size]
test = data[train_size:, :, :]
test_label = labels[train_size:]

46176


In [12]:
train.shape
train_label.shape
train_label[0]

104

In [13]:
test.shape
test_label.shape

(20000,)

In [14]:
class TrainDataset(Dataset):
  
    def __init__(self, train, label):
        self.train = train
        self.label = label

    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        train = torch.Tensor(self.train[idx])
        train = train.transpose(0,1)
#         train = train.to(DEVICE)
        label = self.label[idx]
#         label = torch.Tensor(label)
#         label = label.to(DEVICE)
        return(train, label)

In [15]:
class TestDataset(Dataset):
  
  def __init__(self, test):
    self.test = test
  
  def __len__(self):
    return self.test.shape[0]
  
  def __getitem__(self, idx):
    test = torch.Tensor(self.test[idx])
    test = test.to(DEVICE)
#     test = test.unsqueeze(0)
    return test

In [16]:
train_dataset = TrainDataset(train, train_label)
test_dataset = TestDataset(test)
train_batch_size = 100
test_batch_size = 100
train_loader = DataLoader(train_dataset,
                        batch_sampler=BatchSampler(RandomSampler(train_dataset), train_batch_size, False))
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)

In [17]:
# iter(train_loader).next()
# train_dataset[0][0].shape

In [18]:
class DetectionModel(nn.Module):
    
    def __init__(self,vocab_size,embed_size,hidden_size, nlayers):
        super(DetectionModel,self).__init__()
        self.vocab_size=vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers, bidirectional=True) # Recurrent network
        self.scoring = nn.Linear(hidden_size * 2,vocab_size) # Projection layer
        
    def forward(self, seq_batch):
        batch_size = seq_batch.size(1)
        embed = seq_batch #L x N x E
        hidden = None
        output_lstm,hidden = self.rnn(embed,hidden) #L x N x H
        output_lstm_flatten = output_lstm.view(-1,self.hidden_size * 2) #(L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x V
        return output_flatten.view(-1,batch_size,self.vocab_size)

In [19]:
def train_epoch(model, optimizer, train_loader):
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)
    batch_id=0
    for inputs,targets in train_loader:
        batch_loss = []
        batch_id+=1
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        outputs = model(inputs) # 3D
        outputs = outputs[:, -1, :] # pull out the last layer
#         print(outputs.shape)
        loss = criterion(outputs,targets) # Loss of the flattened outputs
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        torch.cuda.empty_cache()
        if batch_id % 20 == 0:
            lpw = np.mean(batch_loss)
            print("At batch",batch_id)
            print("Training loss :",lpw)

    return model
    

In [20]:
langcount = 176
model = DetectionModel(langcount,40,256,3)
model = model.to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=1e-6)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [ ]:
for i in range(50):
    train_epoch(model, optimizer, train_loader)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


At batch 20
Training loss : 5.15641307831
At batch 40
Training loss : 5.18907976151
At batch 60
Training loss : 5.17420768738
At batch 80
Training loss : 5.18040323257
At batch 100
Training loss : 5.19090080261
At batch 120
Training loss : 5.17324447632
At batch 140
Training loss : 5.17131710052
At batch 160
Training loss : 5.16812801361
At batch 180
Training loss : 5.17479133606
At batch 200
Training loss : 5.17038917542
At batch 220
Training loss : 5.17346000671
At batch 240
Training loss : 5.17114830017
At batch 260
Training loss : 5.17835140228
At batch 280
Training loss : 5.17423582077
At batch 300
Training loss : 5.17857170105
At batch 320
Training loss : 5.17973613739
At batch 340
Training loss : 5.1794090271
At batch 360
Training loss : 5.16720819473
At batch 380
Training loss : 5.16895103455
At batch 400
Training loss : 5.17306041718
At batch 420
Training loss : 5.16647815704
At batch 440
Training loss : 5.17475175858
At batch 460
Training loss : 5.17234134674
At batch 20
Trai

In [19]:
torch.cuda.empty_cache()

In [21]:
print(torch.backends.cudnn.version())

None


In [22]:
print(torch.cuda.is_available())

True
